<a href="https://colab.research.google.com/github/Thireshsidda/Azure-mlops-CICD-pipeline/blob/main/crewai_example_job_application_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Crew to Tailor Job Applications

If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 --quiet
```

In [1]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain_groq

  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.50.0 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [2]:
import os
from getpass import getpass

from crewai import Agent,Task,Crew,Process,LLM
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

In [4]:
SERPER_API_KEY = getpass("Your serper api key")
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
GROQ_API_KEY = getpass("Your Groq api key")
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

Your serper api key··········
Your Groq api key··········


**Note**:
- The video uses `gpt-4o`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4o` when you run the notebook _locally_ (using `gpt-4o` will not work on the platform)
- Thank you for your understanding!

In [6]:
# import os
# from utils import get_openai_api_key, get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [45]:
# llm = ChatGroq(
#     model="qwen-2.5-32b",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     # other params...
# )

# print(llm.invoke("hi"))

llm = LLM(
    model="groq/gemma2-9b-it",
    temperature=0.7
)

## crewAI Tools

In [26]:
!pip install langchain_huggingface

In [46]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [47]:
read_resume = FileReadTool(file_path='./fake_resume.md')
# semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

In [48]:
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md',
    config=dict(
        # llm=dict(
        #     provider="groq", # or google, openai, anthropic, llama2, ...
        #     config=dict(
        #     model="qwen-2.5-32b",
        #         # temperature=0.5,
        #         # top_p=1,
        #         # stream=true,
        #     ),
        # ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [49]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

**Thiresh Sidda**

1234 Elm Street | Bengaluru, Karnataka, India | +91 98765 43210 | thiresh.sidda@email.com | [LinkedIn](https://www.linkedin.com/in/thireshsidda) | [GitHub](https://github.com/thireshsidda)

---

**Professional Summary**

Innovative software engineer with over 8 years of experience in designing and implementing scalable applications. Proficient in multiple programming languages, including Python, Java, and C++. Demonstrated expertise in full-stack development, cloud computing, and leading cross-functional teams to deliver robust software solutions. Skilled in data analysis and machine learning, with a strong foundation in data science principles.

---

**Technical Skills**

- **Programming Languages:** Python, Java, C++, JavaScript
- **Web Technologies:** React, Angular, Node.js, HTML, CSS
- **Databases:** MySQL, PostgreSQL, MongoDB
- **Data Science & Analytics:** Pandas, NumPy, Scikit-learn, TensorFlow
- **Cloud Platforms:** AWS, Azure
- **DevOps Tools:** Docker, Kubernetes, Jenkins
- **Methodologies:** Agile, Scrum, Test-Driven Development (TDD)

---

**Professional Experience**

**Senior Software Engineer**  
*Tech Innovators Pvt. Ltd., Bengaluru, India*  
*April 2020 – Present*

- Led the development of a microservices-based architecture, enhancing system scalability and reducing downtime by 30%.
- Mentored a team of 6 junior developers, resulting in a 25% improvement in code quality and a 15% reduction in bug reports.
- Collaborated with product managers to design and implement new features, increasing user engagement by 20%.
- Applied machine learning algorithms to user data, enhancing personalization features and boosting customer retention by 18%.

**Software Engineer**  
*Creative Solutions Inc., Bengaluru, India*  
*June 2016 – March 2020*

- Developed and maintained web applications using React and Node.js, leading to a 35% increase in user satisfaction.
- Implemented CI/CD pipelines with Jenkins and Docker, reducing deployment times by 40%.
- Optimized database queries, improving application performance by 25%.
- Conducted data analysis to identify user behavior patterns, informing feature development and improving user engagement metrics.
---

**Education**

**Bachelor of Technology in Computer Science**  
*National Institute of Technology, Karnataka, India*  
*Graduated: May 2011*  
- GPA: 8.8/10
- Relevant Coursework: Data Structures, Algorithms, Database Management Systems, Software Engineering

---

**Certifications**

- AWS Certified Solutions Architect – Associate
- Certified ScrumMaster (CSM)
- Oracle Certified Professional, Java SE 8 Programmer
- IBM Data Science Professional Certificate

---

**Projects**

**Real-Time Chat Application**

- Developed a chat application using Socket.io and Node.js, supporting real-time messaging for over 10,000 concurrent users.
- Integrated MongoDB for message storage and retrieval, ensuring data consistency and reliability.

**E-Commerce Recommendation System**

- Designed a recommendation engine using collaborative filtering techniques in Python, increasing average user purchase by 12%.
- Deployed the system on AWS, ensuring scalability and high availability.

---

**Professional Affiliations**

- Member, Association for Computing Machinery (ACM)
- Member, IEEE Computer Society

---


## Creating Agents

In [50]:
# researcher = Agent(
#     role = "Researcher",
#     goal='Pioneer revolutionary advancements in {subject_area}',
#     backstory=(
#     "As a visionary researcher, your insatiable curiosity drives you"
#     "to delve deep into emerging fields. With a passion for innovation"
#     "and a dedication to scientific discovery, you seek to"
#     "develop technologies and solutions that could transform the future."),
#     llm = llm,
#     max_iter = 5,
#     tools = [search_tool],
#     allow_delegation=True,
#     verbose=True
# )

In [51]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm = llm,
    max_iter = 5,
    allow_delegation=True,
    verbose=True
)

In [52]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = llm,
    max_iter = 5,
    allow_delegation=True,
    verbose=True
)

In [53]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm = llm,
    max_iter = 5,
    allow_delegation=True,
    verbose=True
)

In [54]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = llm,
    max_iter = 5,
    allow_delegation=True,
    verbose=True
)

## Creating Tasks

In [55]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [56]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [57]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [58]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [59]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [60]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4136800741',
    'github_url': 'https://github.com/thireshsidda',
    'personal_writeup': """Thiresh Sidda is a seasoned software engineer with over 8 years of experience in designing and implementing scalable applications. Proficient in multiple programming languages, including Python, Java, and C++, he has demonstrated expertise in full-stack development, cloud computing, and leading cross-functional teams to deliver robust software solutions. With a strong foundation in data science principles, Thiresh has successfully applied machine learning algorithms to enhance personalization features, boosting customer retention by 18%. He holds a Master of Technology in Computer Science from the Indian Institute of Technology, Delhi, and is AWS Certified Solutions Architect – Associate. Thiresh is passionate about leveraging technology to drive innovation and growth, making him an ideal candidate for roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [61]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 259c6187-d8d6-4d52-b571-99a61362bb3e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: adaa55ec-8ab6-492f-9f0d-2fa138f1f168
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: adaa55ec-8ab6-492f-9f0d-2fa138f1f168
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
└── 📋 Task: 5a0658e3-ef1f-4834-bdf7-81cec83a4eea
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: adaa55ec-8ab6-492f-9f0d-2fa138f1f168
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
└── 📋 Task: 5a0658e3-ef1f-4834-bdf7-81cec83a4eea
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: adaa55ec-8ab6-492f-9f0d-2fa138f1f168
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
└── 📋 Task: 5a0658e3-ef1f-4834-bdf7-81cec83a4eea
       Status: Executing Task...
    └── 🤖 Agent: Personal Profiler for Engineers
            Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: adaa55ec-8ab6-492f-9f0d-2fa138f1f168
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
└── 📋 Task: 5a0658e3-ef1f-4834-bdf7-81cec83a4eea
       Assigned to: Personal Profiler for Engineers
       Status: ❌ Failed
    └── 🤖 Agent: Personal Profiler for Engineers
            Status: In Progress
        ├── 🔧 Using Read website content (13)
        ├── 🔧 Used Read a file's content (2)
        ├── 🔧 Used Search the internet with Serper (5)
        ├── 🔧 Used Search a MDX's content (2)
        ├── 🔧 Used Read website content (13)
        └── ❌ LLM Failed

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for     │
│  model `gemma2-9b-it` in organization `org_01jq3tppe5fcmbahxdzqjbhx4s` service tier `on_demand` on tokens per   │
│  minute (TPM): Limit 15000, Used 16606, Requested 7304. Please try again in 35.643s. Need more tokens? Upgrade  │
│  to Dev Tier today at                                                                                           │
│  https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
├── 📋 Task: adaa55ec-8ab6-492f-9f0d-2fa138f1f168
│      Assigned to: Tech Job Researcher
│      Status: ❌ Failed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
└── 📋 Task: 5a0658e3-ef1f-4834-bdf7-81cec83a4eea
       Assigned to: Personal Profiler for Engineers
       Status: ❌ Failed
    └── 🤖 Agent: Personal Profiler for Engineers
            Status: In Progress
        ├── 🔧 Using Read website content (13)
        ├── 🔧 Used Read a file's content (2)
        ├── 🔧 Used Search the internet with Serper (5)
        ├── 🔧 Used Search a MDX's content (2)
        ├── 🔧 Used Read website content (13)
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: adaa55ec-8ab6-492f-9f0d-2fa138f1f168                                                                     │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

KeyboardInterrupt: 

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

- Dislplay the generated `interview_materials.md` file.

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))